# 实践作业  
作业说明：已给出关键代码提示，可根据自身能力选择基础或进阶作业完成，想多尝试也可以两部分都完成。  
 

### 基础作业  
 根据教案中的学习代码，尝试做以下的调整，需要有尝试的过程代码和结果的文字总结。  


**1. 尝试不同的LoRA配置参数，对比性能差异**  
**概述**：在实际开发中，不同的LoRA配置参数会影响模型的性能，尝试不同的参数配置确定模型的最佳性能。  
**方法**：选则固定的评估指标，然后排列各种参数配置，观察不同参数配置下的评估指标的变化，当某个参数配置的评估指标最优时，则认为该参数配置最优  
- rank (r): 低秩矩阵的维度  
典型范围: 2-256，值越大表示适配能力越强，但也可能过拟合  
- alpha (α): 缩放因子  
控制适配强度，通常设置为rank的倍数(如2×rank)  
- dropout: 防止过拟合  
典型值: 0-0.2  
- target_modules: 应用LoRA的模块  
常见选择: query/key/value层或全连接层

**2. 设计并实现自定义的评估指标**  
- **概述**：在LLM中，还有许多不同评估角度的评估指标，例如BULE，Rouge，N-Gram，可手动尝试不同的指标对模型性能的影响  
- **方法**：例如使用BULE，Rouge作为评估指标，然后进行训练，训练后进行实际测评（文本生成、翻译等任务），观察微调后的模型性能是否有改善，若有，则认为这些评估指标有效

In [ ]:
#你的代码过程

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_chinese import Rouge
import jieba
import numpy as np

class TextEvaluator:
    def __init__(self):
        """初始化评估器"""
        self.rouge = Rouge()
        self.smooth = SmoothingFunction()
    
    def calculate_bleu(self, reference, candidate):
        """
        计算BLEU分数
        
        参数:
            reference (str): 参考文本
            candidate (str): 候选文本
            
        返回:
            float: BLEU分数
        """
        # 将文本分词
        reference_tokens = list(jieba.cut(reference))
        candidate_tokens = list(jieba.cut(candidate))
        
        # 计算BLEU分数
        try:
            score = sentence_bleu([reference_tokens], candidate_tokens, 
                                smoothing_function=self.smooth.method1)
            return score
        except Exception as e:
            print(f"计算BLEU分数时出错: {e}")
            return 0.0
    
    def calculate_rouge(self, reference, candidate):
        """
        计算ROUGE分数
        
        参数:
            reference (str): 参考文本
            candidate (str): 候选文本
            
        返回:
            dict: 包含ROUGE-1、ROUGE-2和ROUGE-L分数的字典
        """
        try:
            scores = self.rouge.get_scores(candidate, reference)[0]
            return {
                'rouge-1': scores['rouge-1']['f'],
                'rouge-2': scores['rouge-2']['f'],
                'rouge-l': scores['rouge-l']['f']
            }
        except Exception as e:
            print(f"计算ROUGE分数时出错: {e}")
            return {'rouge-1': 0.0, 'rouge-2': 0.0, 'rouge-l': 0.0}
    
    def evaluate_text(self, reference, candidate):
        """
        综合评估文本质量
        
        参数:
            reference (str): 参考文本
            candidate (str): 候选文本
            
        返回:
            dict: 包含所有评估指标的字典
        """
        # 计算BLEU分数
        bleu_score = self.calculate_bleu(reference, candidate)
        
        # 计算ROUGE分数
        rouge_scores = self.calculate_rouge(reference, candidate)
        
        # 返回所有评估指标
        return {
            'bleu': bleu_score,
            **rouge_scores
        }

# 测试代码
if __name__ == "__main__":
    # 创建评估器实例
    evaluator = TextEvaluator()
    
    # 测试文本
    reference = "这是一个测试文本，用于评估文本生成质量。"
    candidate = "这是一个测试文本，用来评估生成文本的质量。"
    
    # 计算评估指标
    scores = evaluator.evaluate_text(reference, candidate)
    
    # 打印结果
    print("\n评估结果:")
    print(f"BLEU分数: {scores['bleu']:.4f}")
    print(f"ROUGE-1分数: {scores['rouge-1']:.4f}")
    print(f"ROUGE-2分数: {scores['rouge-2']:.4f}")
    print(f"ROUGE-L分数: {scores['rouge-l']:.4f}")

### 进阶作业  
**将微调方法应用到其他语言**  
- **概述**：微调特点成本低、效用高，基座大模型可以通过微调满足特定领域的任务，微调成为是业界最常用的方法，可使用挂载好的韩语数据集，学习教案中的阿拉伯语微调教程，完成大模型学习韩语的任务  
- **方法**：在完成基础作业的基础上（确定了Lora配置参数与评估指标）， 给大模型喂入韩语，完成微调

In [2]:
from typing import List, Dict, Set

def tokenize_text(text: str, lang: str = None) -> List[str]:
    """根据不同语言使用相应的分词器
    
    Args:
        text: 要分词的文本
        lang: 语言代码（如果为None则自动检测）
    
    Returns:
        分词后的词语列表
    """
    if not text:
        return []
        
    # 如果没有指定语言，进行语言检测
    if lang is None:
        lang = detect_language(text)
    
    # 根据语言选择分词方法
    if lang == 'zh':
        # 中文使用jieba分词
        return [word for word in jieba.cut(text) if word.strip()]
    elif lang == 'th':
        # 泰语使用pythainlp分词
        return [word for word in thai_tokenize(text) if word.strip()]
    elif lang == 'ko':
        # 韩语分词
        if mecab:
            try:
                # 使用mecab进行分词，过滤掉助词等
                return [word for word, pos in mecab.pos(text) 
                       if word.strip() and not pos.startswith('J')]
            except Exception:
                pass
        
        # 如果mecab不可用或失败，使用基本分词
        words = []
        current_word = ''
        for char in text:
            if '\uAC00' <= char <= '\uD7AF' or char.isalnum():
                current_word += char
            else:
                if current_word:
                    words.append(current_word)
                current_word = ''
                if not char.isspace():
                    words.append(char)
        if current_word:
            words.append(current_word)
        return [w for w in words if w.strip()]
    
    elif lang == 'ru':
        # 俄语分词 - 使用基本的razdel分词
        return [token.text for token in ru_tokenize(text) 
                if token.text.strip() and not all(c in '.,!?;:()[]{}' for c in token.text)]
    
    if lang == 'ar':
        # 标准化阿拉伯文本中的hamza字符(ء/أ/إ/ؤ/ئ)为统一形式，原因是阿拉伯语的变音符号会影响分词结果
        text = araby.normalize_hamza(text)
        # 标准化alef字符(آ/إ/أ/ا)为基本的alef(ا)，原因是阿拉伯语的alef字符会影响分词结果
        text = araby.normalize_alef(text)
        # 使用araby库对阿拉伯文本进行分词
        words = araby.tokenize(text)
        # 过滤分词结果:
        # 1. 使用strip()去除首尾空白字符
        # 2. 排除只包含标点符号的词
        # 3. 返回过滤后的词列表
        return [word for word in words if word.strip() and not all(c in '.,!?;:()[]{}' for c in word)]
    else:
        # 其他语言使用空格分词
        return [word for word in text.split() if word.strip()]

def extract_domain_terms(domain_texts: List[str], general_texts: List[str] = None, top_n: int = 2000) -> List[str]:
    """使用改进的多语言分词方法提取领域术语"""
    print("开始提取领域术语...")
    
    # 按语言分组处理文本
    lang_texts = defaultdict(list)
    for text in domain_texts:
        lang = detect_language(text)
        if lang != 'unknown':
            lang_texts[lang].append(text)
    
    print("\n文本语言分布:")
    for lang, texts in lang_texts.items():
        print(f"- {lang}: {len(texts)} 条")
    
    # 分语言处理并提取术语
    all_terms = []
    for lang, texts in lang_texts.items():
        print(f"\n处理{lang}语言文本...")
        
        # 使用语言专属工具提取术语
        lang_terms = set()
        for text in tqdm(texts, desc=f"处理{lang}语言文本"):
            terms = extract_domain_terms_by_language(text, lang)
            lang_terms.update(terms)
        
        # 将术语集合转换为列表,方便后续排序操作
        lang_terms = list(lang_terms)

        # 计算每个术语在文本中出现的频率
        # 1. 遍历每个文本
        # 2. 对每个文本重新提取术语
        # 3. 使用Counter统计所有术语的频率
        term_freq = Counter(t for text in texts for t in extract_domain_terms_by_language(text, lang))

        # 对术语列表进行排序:
        # 1. 首要排序依据是术语出现频率(term_freq[x])
        # 2. 次要排序依据是术语长度(len(x))
        # reverse=True表示按降序排列,即频率高的和长度长的排在前面
        lang_terms.sort(key=lambda x: (term_freq[x], len(x)), reverse=True)
        
        # 根据语言数量平均分配术语数量配额
        # 1. top_n是总的期望术语数量
        # 2. len(lang_texts)是语言种类数
        # 3. 对每种语言,只取配额内的高频长术语
        # //是整除运算符,用于计算每种语言分配的术语数量配额
        # 例如:如果top_n=2000,有4种语言,则每种语言分配2000//4=500个术语
        # 这里的:是切片操作符,表示从列表开头取到指定位置
        # //是整除运算符,例如10//3=3
        # 所以lang_terms[:top_n // len(lang_texts)]表示:
        # 1. 先计算top_n除以语言数量的整除结果n
        # 2. 然后从lang_terms列表中取前n个元素
        selected_terms = lang_terms[:top_n // len(lang_texts)]
        all_terms.extend(selected_terms)
        
        print(f"{lang}语言提取了 {len(selected_terms)} 个术语")
        if selected_terms:
            print(f"{lang}语言术语示例:")
            for term in selected_terms[:5]:
                print(f"  - {term}")
    
    return all_terms



def extract_domain_terms_by_language(text: str, lang: str) -> List[str]:
    """使用语言专属工具提取领域术语"""
    terms = []
    
    if lang == 'ko':
        try:
            # 使用KoNLPy的Mecab分析器
            if mecab:
                # 提取名词和专有名词
                pos_tags = mecab.pos(text)
                terms = [word for word, pos in pos_tags 
                        if pos.startswith('NN') or pos.startswith('NNP')]
        except Exception as e:
            print(f"韩语处理失败: {e}")
    
    elif lang == 'ru':
        try:
            # 使用pymorphy2进行形态分析
            if morph:
                words = [t.text for t in ru_tokenize(text)]
                for word in words:
                    parsed = morph.parse(word)[0]
                    # 提取名词和专有名词
                    if 'NOUN' in parsed.tag or 'Name' in parsed.tag:
                        terms.append(word)
        except Exception as e:
            print(f"俄语处理失败: {e}")
    
    elif lang == 'th':
        try:
            # 使用pythainlp进行词性标注
            words = thai_tokenize(text)
            pos_tags = pythainlp.tag.pos_tag(words)
            # 提取名词和专有名词
            terms = [word for word, pos in pos_tags 
                    if pos.startswith('N') or pos.startswith('PROPN')]
        except Exception as e:
            print(f"泰语处理失败: {e}")
    
    elif lang == 'ar':
        try:
            # 使用pyarabic进行基本处理
            text = araby.strip_tashkeel(text)  # 移除变音符号
            words = araby.tokenize(text)
            # 提取可能的术语（长度大于3的词）
            terms = [w for w in words if len(w) > 3 and not any(c.isdigit() for c in w)]
        except Exception as e:
            print(f"阿拉伯语处理失败: {e}")
    
    return terms

